In [127]:
# Dependencies
import requests
from config import api_key
import pandas as pd
import json
import os
from sqlalchemy import create_engine
import numpy as np
from pprint import pprint 
from sqlalchemy.orm import Session

In [128]:
wiki_data = pd.read_csv("Resources/wiki_movie_plots.csv")

wiki_data.head()

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,Unnamed: 8
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr...",NaN
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov...",NaN
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed...",NaN
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...,NaN
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...,NaN


In [129]:
# #dropping all un-needed column
wiki_data.drop(['Director', 'Cast', 'Genre', 'Director', 'Unnamed: 8', 'Wiki Page'], axis=1, inplace=True)
wiki_data 

,Release Year,Title,Origin/Ethnicity,Plot
0,1901,Kansas Saloon Smashers,American,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,The earliest known adaptation of the classic f...
...,...,...,...,...
34887,2014,The Water Diviner,Turkish,"The film begins in 1919, just after World War ..."
34888,2017,Çalgı Çengi İkimiz,Turkish,"Two musicians, Salih and Gürkan, described the..."
34889,2017,Olanlar Oldu,Turkish,"Zafer, a sailor living with his mother Döndü i..."
34890,2017,Non-Transferable,Turkish,The film centres around a young woman named Am...


In [130]:
# Count duplicate in a DataFrame
wiki_data.duplicated().sum()

43

In [131]:
# checking for duplicate in a DataFrame
wiki_data.drop_duplicates(inplace=True)

In [132]:
# checking Count of duplicates in a DataFrame after dropping the duplicates
wiki_data.duplicated().sum()

0

In [133]:
#checking the datatype
wiki_data.dtypes

Release Year        object
Title               object
Origin/Ethnicity    object
Plot                object
dtype: object

In [134]:
#looking all the unwanted information in the Release Year so that we can merge the data
wiki_data['Release Year'].unique()

array(['1901', '1902', '1903', '1904', '1905', '1906', '1907', '1908',
       '1909', '1910', '1911', '1912', '1913', '1914', '1915', '1916',
       '1917', '1918', '1919', '1920', '1921', '1922', '1923', '1924',
       '1925', '1926', '1927', '1928', '1929', '1930', '1931', '1932',
       '1933', '1934', '1935', '1936', '1937', '1938', '1939', '1940',
       '1941', '1942', '1943', '1944', '1945', '1946', '1947', '1948',
       '1949', '1950', '1951', '1952', '1953', '1954', '1955', '1956',
       '1957', '1958', '1959', '1960', '1961', '1962', '1963', '1964',
       '1965', '1966', '1967', '1968', '1969', '1970', '1971', '1972',
       '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980',
       '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988',
       '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996',
       '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012',
      

In [135]:
imdb_data = pd.read_csv("Resources/imdb_top_1000.csv")
imdb_data.head()


,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"


In [136]:
# #dropping all un-needed columns from te imdb_data table
imdb_data.drop(['Poster_Link', 'Certificate'], axis=1, inplace=True)
#movies_merge_df 
imdb_data.head()

,Series_Title,Released_Year,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,The Shawshank Redemption,1994,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
1,The Godfather,1972,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
2,The Dark Knight,2008,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
3,The Godfather: Part II,1974,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
4,12 Angry Men,1957,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"


In [137]:
# Count duplicate in a DataFrame
imdb_data.duplicated().sum()

0

In [138]:
imdb_data.dtypes

Series_Title      object
Released_Year     object
Runtime           object
Genre             object
IMDB_Rating      float64
Overview          object
Meta_score       float64
Director          object
Star1             object
Star2             object
Star3             object
Star4             object
No_of_Votes        int64
Gross             object
dtype: object

In [139]:
#inspecting and verifying the data available in the imdb before joining the data
imdb_data.Released_Year.unique()

array(['1994', '1972', '2008', '1974', '1957', '2003', '1993', '2010',
       '1999', '2001', '1966', '2002', '1990', '1980', '1975', '2020',
       '2019', '2014', '1998', '1997', '1995', '1991', '1977', '1962',
       '1954', '1946', '2011', '2006', '2000', '1988', '1985', '1968',
       '1960', '1942', '1936', '1931', '2018', '2017', '2016', '2012',
       '2009', '2007', '1984', '1981', '1979', '1971', '1963', '1964',
       '1950', '1940', '2013', '2005', '2004', '1992', '1987', '1986',
       '1983', '1976', '1973', '1965', '1959', '1958', '1952', '1948',
       '1944', '1941', '1927', '1921', '2015', '1996', '1989', '1978',
       '1961', '1955', '1953', '1925', '1924', '1982', '1967', '1951',
       '1949', '1939', '1937', '1934', '1928', '1926', '1920', '1970',
       '1969', '1956', '1947', '1945', '1930', '1938', '1935', '1933',
       '1932', '1922', '1943', 'PG'], dtype=object)

In [141]:
#cleaning the wiki_data to removed the "s messy and his clothes are torn. He looks ......" text from the year released column
wiki_data = wiki_data[wiki_data["Release Year"].str.contains("s messy and his clothes are torn. He looks at his army who watch in shock.")==False]

#cleaning the wiki_data to removed the "Rama is praying for safety but no help is...." text from the year released column
wiki_data = wiki_data[wiki_data["Release Year"].str.contains("Rama is praying for safety but no help is given to him. Rama is presumed dead but Sita prays and the chakra grows bigger. With great difficulty the hurt Rama crawls towards the distant chakra. He is moving very slowly and screaming in pain. His wounds are getting much worse and his bones are broken. He is struggling to live. Ravana is about to kill the weak Rama. Rama is stopping and crying because he was too weak to save Sita. Everyone else knew that Rama was about to die and failed to rescue his wife after a 4-year battle.But before Ravana could kill the wounded Rama")==False]

#cleaning the wiki_data to removed the "With the battle over" text from the year released column
wiki_data = wiki_data[wiki_data["Release Year"].str.contains("With the battle over")==False]

wiki_data.head()

,Release Year,Title,Origin/Ethnicity,Plot
0,1901,Kansas Saloon Smashers,American,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,The earliest known adaptation of the classic f...


In [142]:
#Change the Released Year datatype in the wiki_data_clean from object to Int64 datatype so that we can merge the two datasets
wiki_data['Release Year'] = pd.to_numeric(wiki_data['Release Year'])
wiki_data.dtypes

Release Year         int64
Title               object
Origin/Ethnicity    object
Plot                object
dtype: object

In [143]:
#cleaning the imdb_data to removed the "PG" text from the year released column
imdb_data = imdb_data[imdb_data["Released_Year"].str.contains("PG")==False]

#Change the Released_Year datatype from object to Int64 datatype so that we can merge the two datasets
imdb_data['Released_Year'] = pd.to_numeric(imdb_data['Released_Year'])

#imdb_data.dtypes
imdb_data.dtypes

Series_Title      object
Released_Year      int64
Runtime           object
Genre             object
IMDB_Rating      float64
Overview          object
Meta_score       float64
Director          object
Star1             object
Star2             object
Star3             object
Star4             object
No_of_Votes        int64
Gross             object
dtype: object

In [151]:
#joining the imdb_data table and the wiki_data tables to create a new table
movies_merge_df = pd.merge(imdb_data, wiki_data, left_on = ["Series_Title", 'Released_Year'], right_on = ["Title", "Release Year"], how = "left")

movies_merge_df.head()

,Series_Title,Released_Year,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross,Release Year,Title,Origin/Ethnicity,Plot
0,The Shawshank Redemption,1994,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469",1994.0,The Shawshank Redemption,American,"In 1947 Portland, Maine, banker Andy Dufresne ..."
1,The Godfather,1972,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411",1972.0,The Godfather,American,"In 1945, at his daughter Connie's wedding, Vit..."
2,The Dark Knight,2008,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444",2008.0,The Dark Knight,British,A gang of criminals rob a Gotham City mob bank...
3,The Godfather: Part II,1974,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000",NaN,NaN,NaN,NaN
4,12 Angry Men,1957,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000",1957.0,12 Angry Men,American,In a New York City courthouse a jury commences...


In [152]:
# #dropping all un-needed column
movies_merge_df.drop(['Release Year', 'Title'], axis=1, inplace=True)
#movies_merge_df 
movies_merge_df.head()



,Series_Title,Released_Year,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross,Origin/Ethnicity,Plot
0,The Shawshank Redemption,1994,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469",American,"In 1947 Portland, Maine, banker Andy Dufresne ..."
1,The Godfather,1972,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411",American,"In 1945, at his daughter Connie's wedding, Vit..."
2,The Dark Knight,2008,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444",British,A gang of criminals rob a Gotham City mob bank...
3,The Godfather: Part II,1974,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000",NaN,NaN
4,12 Angry Men,1957,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000",American,In a New York City courthouse a jury commences...


In [153]:
#identify or check if there is anymore duplicates
movies_merge_df.duplicated().sum()


0

In [155]:
#identify the binary columns to see all the false values
movies_merge_df.isnull()

,Series_Title,Released_Year,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross,Origin/Ethnicity,Plot
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1006,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False
1007,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False
1008,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1009,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False


In [160]:
#dropping all rows with NAN fields
movies_merge_df.dropna(axis=0, how= "any", inplace=True)
movies_merge_df.head()

,Series_Title,Released_Year,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross,Origin/Ethnicity,Plot
0,The Shawshank Redemption,1994,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469",American,"In 1947 Portland, Maine, banker Andy Dufresne ..."
1,The Godfather,1972,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411",American,"In 1945, at his daughter Connie's wedding, Vit..."
2,The Dark Knight,2008,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444",British,A gang of criminals rob a Gotham City mob bank...
4,12 Angry Men,1957,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000",American,In a New York City courthouse a jury commences...
5,The Lord of the Rings: The Return of the King,2003,201 min,"Action, Adventure, Drama",8.9,Gandalf and Aragorn lead the World of Men agai...,94.0,Peter Jackson,Elijah Wood,Viggo Mortensen,Ian McKellen,Orlando Bloom,1642758,"377,845,905",American,"Many years ago, two Hobbits, Smeagol and Déago..."


In [158]:
#create a non-sql database
# Import and establish Base for which classes will be constructed 
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [159]:
# Import modules to declare columns and column data types
from sqlalchemy import Column, Integer, String, Float

In [161]:
# Create Surfer and Board classes
#Series_Title	Released_Year	Runtime	Genre	IMDB_Rating	Overview	Meta_score	Director	Star1	Star2	Star3	Star4	No_of_Votes	Gross	Origin/Ethnicity	Plot# ----------------------------------
class movies_merge_df(Base):
    __tablename__ = 'movies_info'
    id = Column(Integer, primary_key=True)
    Series_Title = Column(String(10))
    Released_Year = Column(Integer)
    Runtime = Column(String(10))
    Genre = Column(String(50))
    IMDB_Rating = Column(String(5))
    Overview = Column(String(255))
    Meta_score = Column(Integer)
    Director = Column(String(20))
    Star1 = Column(String(20))
    Star2 = Column(String(20))
    Star3 = Column(String(20))
    Star4 = Column(String(20))
    No_of_Votes = Column(Integer)
    Gross = Column(Integer)
    Title = Column(String(50))
    Origin_Ethnicity = Column(String(50))    
    Plot = Column(String(100))

In [165]:
# Establish Connection
engine = create_engine('Resources/movies_info.sqlite')
Base.metadata.create_all(engine)

ArgumentError: Could not parse SQLAlchemy URL from string 'Resources/movies_info.sqlite'

In [163]:
# Create both the Surfer and movies_info table within the database
Base.metadata.create_all(conn)

NameError: name 'conn' is not defined

In [ ]:
# To push the objects made and query the server we use a Session object
from sqlalchemy.orm import Session
session = Session(bind=engine)

In [ ]:
# Query the database and collect all of the surfers in the Surfer table
movies_list = session.query(movies_info)
for movie in movies_list:
    print(movie.Title)
    print(movie.Director)
    print(movie.Plot)